In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','size']

In [3]:
start = 20130101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
print(df_d.columns)
print(df_d.shape)

select * from stock_day where date>=20130101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(2304627, 11)


In [5]:
start = 20130101
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
print(start_date)
df_rs_sp1 = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,
                                                                    start_date:],:].sort_index()

2013-01-01


In [6]:
df_d['next_low'] = df_d.groupby('code')['low'].shift(-1)

In [10]:
df_r = df_rs_sp1.join(df_d[['close','next_low']],how='inner')


In [41]:
print(df_r.head())
print(df_d.head())
print(df_r.r.agg(agg_operations[:4]))
df_r.r.dropna().agg('size')

                         buy_at        max  max_idx  idx  is_selled  \
code      date                                                        
000001.SZ 2013-01-04  578.04454  592.15201    663.0  661       True   
          2013-01-07  589.61990  592.15201    663.0  662       True   
          2013-01-08  577.32108  618.19657    667.0  663       True   
          2013-01-09  573.70378  618.19657    667.0  664       True   
          2013-01-10  574.06551  618.19657    667.0  665       True   

                        sell_at         r      close   next_low  
code      date                                                   
000001.SZ 2013-01-04  560.31977 -0.030663  578.40627  574.42724  
          2013-01-07  560.31977 -0.049693  589.61990  573.70378  
          2013-01-08  619.28176  0.072682  578.76800  571.53340  
          2013-01-09  619.28176  0.079445  573.70378  571.17167  
          2013-01-10  619.28176  0.078765  574.06551  558.87285  
                           open       hi

2164955

In [79]:
print('low/open:\n',(df_d['low']/df_d['open']).agg(['mean','median','std']),'\n'+'-'*10)
print('next_low/close:\n',(df_d['next_low']/df_d['close']).agg(['mean','median','std']),'\n'+'-'*10)
print()

t_open = 0.85
mask_lower_open = (df_r['buy_at']*t_open >= df_r['next_low']) & (df_r.r.notnull())
print(((df_r[mask_lower_open].r+1)/t_open-1).agg(agg_operations[:4]))
print(mask_lower_open.sum())
print()
mask0 = (df_r.r.notnull()) & (df_r.next_low.notnull())
print(df_r[mask0].r.agg(agg_operations[:4]))
print(df_r[mask0].r.notnull().sum())
# print(((df_r[mask_lower_open].r+1)/t_open-1).head(5))
# print(df_r.r.head())

low/open:
 mean      0.982116
median    0.987598
std       0.018588
dtype: float64 
----------
next_low/close:
 mean      0.981086
median    0.986446
std       0.026033
dtype: float64 
----------

mean     -0.039592
median   -0.065336
max       0.631879
min      -0.298809
Name: r, dtype: float64
998

mean       0.002743
median    -0.008555
max       10.329513
min       -0.872727
Name: r, dtype: float64
2160638


In [61]:
df_r[mask0!=mask_lower_open]

,,buy_at,max,max_idx,idx,is_selled,sell_at,r,close,next_low
code,date,,,,,,,,,
600401.SH,2018-02-05,14.71704,14.71704,1364.0,1363,True,2.64152,-0.820513,16.41516,16.41516


In [67]:
df_d.loc[idx['600401.SH','2018-04':'2018-05'],:]

open      high       low     close  vol  amt  \
code      date                                                           
600401.SH 2018-04-02  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-03  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-04  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-09  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-10  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-11  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-12  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-13  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-16  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-17  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-18  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-19  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-20  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-23  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-24  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-25  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-26  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-04-27  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-02  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-03  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-04  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-07  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-08  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-09  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-10  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-11  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-14  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-15  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-16  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-17  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-18  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-21  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-22  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-23  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-24  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-25  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-28  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-29  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-30  16.41516  16.41516  16.41516  16.41516  0.0  0.0   
          2018-05-31  16.41516  16.41516  16.41516  16.41516  0.0  0.0   

                      adj_factor       avg  close0  avg0  pct  next_low  
code      date                                                           
600401.SH 2018-04-02      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-03      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-04      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-09      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-10      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-11      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-12      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-13      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-16      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-17      18.868  16.41516    0.87  0.87  0.0  16.41516  
          2018-04-18      18.